### This notebook demonstrates how to use the BattETL Loader class. 

Note that in order for this notebook to work a BattDB database must be deployed and defined in a `.env` file in this working directory

In [1]:
import os
import json
import pandas as pd
from battetl.extract import Extractor
from battetl.transform import Transformer
from battetl.load import Loader

In [3]:
# Extract and transform the data 
data_directory = os.path.join('..', '..','tests','data','maccor_cycler_data','simple_data')
data_file_path = os.path.join(data_directory,'BG_Maccor_Testdata - 079.txt')
stats_file_path = os.path.join(data_directory,'BG_Maccor_Testdata - 079 [STATS].txt')
schedule_file_paths = [os.path.join(data_directory,'BG_Maccor_Schedule.000')]
paths = [
    data_file_path,
    stats_file_path
]

extractor = Extractor()
extractor.schedule_from_files(schedule_file_paths)
extractor.data_from_files(paths)

transformer = Transformer(timezone='America/Los_Angeles')
transformer.transform_test_data(extractor.raw_test_data)
transformer.transform_cycle_stats(extractor.raw_cycle_stats)

2023-04-07 12:19:02,805 INFO [utils:24] Load environment variables from /Users/zander/Desktop/battetl/examples/submodule_demos/.env
2023-04-07 12:19:02,814 INFO [utils:28] Set logger level to DEBUG
2023-04-07 12:19:02,834 INFO [Extractor:73] Total 1 files
2023-04-07 12:19:02,837 INFO [Extractor:77] Processing procedure files for Maccor
2023-04-07 12:19:02,838 INFO [Extractor:125] Importing Maccor procedure ../../tests/data/maccor_cycler_data/simple_data/BG_Maccor_Schedule.000
2023-04-07 12:19:02,853 INFO [Extractor:252] Identified Maccor procedure steps: {"chg": [2, 7, 12, 17, 22, 27, 32, 37], "dsg": [5, 10, 15, 20, 25, 30, 35], "rst": [1, 3, 6, 8, 11, 13, 16, 18, 21, 23, 26, 28, 31, 33, 36, 38]}
2023-04-07 12:19:02,854 INFO [Extractor:53] Total 2 files
2023-04-07 12:19:02,855 INFO [Extractor:574] Load file path: ../../tests/data/maccor_cycler_data/simple_data/BG_Maccor_Testdata - 079.txt
2023-04-07 12:19:02,856 DEBUG [Extractor:579] header lines: 2
2023-04-07 12:19:02,859 DEBUG [Extra

,cycle,cycle type,test_time_s,maccor_min_current_ma,maccor_min_voltage_mv,reported_charge_capacity_mah,reported_discharge_capacity_mah,reported_charge_energy_mwh,reported_discharge_energy_mwh,acr_ohm,dcir,maccor_charge_thermocouple_start_c,maccor_charge_thermocouple_end_c,maccor_charge_thermocouple_min_c,maccor_charge_thermocouple_max_c,maccor_discharge_thermocouple_start_c,maccor_discharge_thermocouple_end_c,maccor_discharge_thermocouple_min_c,maccor_discharge_thermocouple_max_c,date
0,0,NaN,539.16,0.099794,4.200122,36.577804,NaN,153.244853,NaN,0,0,26.537836,26.621141,26.537836,26.654465,NaN,NaN,NaN,NaN,43916.964748
1,1,NaN,140872.77,0.099794,4.199969,2890.785664,2903.893350,10844.128589,10684.808253,0,0,26.654465,26.587818,26.304579,26.987686,26.487854,26.887722,26.271259,26.904381,43918.588977
2,2,NaN,210878.16,0.099947,4.199969,2857.675195,2860.099388,10750.549863,10517.275027,0,0,26.637804,26.471189,26.371225,26.954365,26.554497,27.087656,26.354565,27.154297,43919.399228
3,3,NaN,246583.54,0.099794,4.199817,2833.653732,2832.018735,10713.034852,10383.109116,0,0,26.871058,26.571159,26.554497,27.170961,26.421207,27.704117,26.221273,27.704117,43919.812493
4,4,NaN,262234.69,0.099947,4.200122,2804.068778,2801.992153,10724.094741,10173.873951,0,0,27.220943,26.671124,26.654465,27.987354,26.521175,29.436874,26.504513,29.436874,43919.993650
5,5,NaN,271566.09,0.099794,4.200122,2778.967842,2778.055297,10774.286333,9936.036711,0,0,27.720778,26.521175,26.521175,31.011431,26.454531,32.686275,26.454531,32.686275,43920.101663
6,6,NaN,277930.22,0.099641,4.199969,2759.421513,2758.559727,10889.422138,9627.696107,0,0,29.770096,26.821075,26.821075,39.401909,26.487854,41.535053,26.471189,41.535053,43920.175333
7,7,NaN,283295.21,0.099947,4.199817,2743.710761,2742.484959,10964.523516,9399.777453,0,0,31.531771,26.671124,26.654465,48.155888,26.637804,51.243423,26.621141,51.243423,43920.237438


In [5]:
# For the Loader we must have a config that defines the test meta data
loader_config = {
    "meta_data": {
        "test_meta": {
            "test_name": "test_test",
            "channel": 10
        },
        "cell": {
            "manufacturer_sn": "0001"
        },
        "cell_meta": {
            "manufacturer": "FakeMN",
            "manufacturer_pn": "1234"
        },
        "schedule_meta": {
            "schedule_name": "fake_schedule.000",
            "cycler_make": "BattGenie",
            "schedule_files": json.dumps(extractor.schedule)
        },
        "cycler": {
            "sn": "0001"
        },
        "cycler_meta": {
            "manufacturer": "BattGenie",
            "model": "Cycler9000"
        }
    }
}

In [6]:
# Initialize loader with the config and create connection to database
loader = Loader(loader_config)


2023-04-07 12:19:13,023 INFO [utils:24] Load environment variables from /Users/zander/Desktop/battetl/examples/submodule_demos/.env
2023-04-07 12:19:13,028 INFO [utils:28] Set logger level to DEBUG
2023-04-07 12:19:13,139 INFO [Loader:334] Created connection to database battdb


In [7]:
# Load the test data to the database. This will create all the associated meta entires if they do not already exist. 
loader.load_test_data(transformer.test_data)

2023-04-07 12:19:16,598 INFO [Loader:371] Move fields to other_detail: dcir, acr, maccor_capacity_mah, es, thermocouple_1_c, maccor_energy_mwh
100%|██████████| 141981/141981 [00:05<00:00, 28137.23it/s]
2023-04-07 12:19:21,706 DEBUG [Loader:149] Dropping column maccor_capacity_mah from DataFrame
2023-04-07 12:19:21,714 DEBUG [Loader:149] Dropping column maccor_energy_mwh from DataFrame
2023-04-07 12:19:21,723 DEBUG [Loader:149] Dropping column es from DataFrame
2023-04-07 12:19:21,729 DEBUG [Loader:149] Dropping column acr from DataFrame
2023-04-07 12:19:21,737 DEBUG [Loader:149] Dropping column dcir from DataFrame
2023-04-07 12:19:21,744 DEBUG [Loader:149] Dropping column thermocouple_1_c from DataFrame
2023-04-07 12:19:21,800 DEBUG [Loader:412] Lookup test_id: None
2023-04-07 12:19:21,807 DEBUG [Loader:450] Lookup cell_type_id: None
2023-04-07 12:19:21,808 INFO [Loader:621] No cell_id exists for {"manufacturer_sn": "0001"}, creating new entry
2023-04-07 12:19:21,810 DEBUG [Loader:450]

141981

In [9]:
''' 
Note that if we try to reload the data it does not overwrite what is already there. It only adds new data determined 
by a row have a unixtime_s value greater than the largest unixtime_s currently in the test_data table.
'''
loader.load_test_data(transformer.test_data)


2023-04-07 12:21:14,432 INFO [Loader:371] Move fields to other_detail: dcir, acr, maccor_capacity_mah, es, thermocouple_1_c, maccor_energy_mwh
100%|██████████| 141981/141981 [00:05<00:00, 27484.76it/s]
2023-04-07 12:21:19,606 DEBUG [Loader:149] Dropping column maccor_capacity_mah from DataFrame
2023-04-07 12:21:19,612 DEBUG [Loader:149] Dropping column maccor_energy_mwh from DataFrame
2023-04-07 12:21:19,621 DEBUG [Loader:149] Dropping column es from DataFrame
2023-04-07 12:21:19,630 DEBUG [Loader:149] Dropping column acr from DataFrame
2023-04-07 12:21:19,641 DEBUG [Loader:149] Dropping column dcir from DataFrame
2023-04-07 12:21:19,649 DEBUG [Loader:149] Dropping column thermocouple_1_c from DataFrame
2023-04-07 12:21:19,672 DEBUG [Loader:412] Lookup test_id: 16
2023-04-07 12:21:19,678 DEBUG [Loader:412] Lookup test_id: 16
2023-04-07 12:21:19,764 INFO [Loader:817] Latest unixtime_s for test_id=16 is 1585572714
2023-04-07 12:21:19,784 INFO [Loader:885] Inserting 0 rows into test_data 

0

In [10]:
# Load cycle stats data to database
loader.load_cycle_stats(transformer.cycle_stats)

2023-04-07 12:21:30,363 INFO [Loader:371] Move fields to other_detail: maccor_charge_thermocouple_end_c, dcir, maccor_min_current_ma, cycle type, maccor_discharge_thermocouple_max_c, maccor_charge_thermocouple_max_c, maccor_discharge_thermocouple_min_c, maccor_charge_thermocouple_min_c, maccor_discharge_thermocouple_start_c, maccor_discharge_thermocouple_end_c, date, maccor_min_voltage_mv, maccor_charge_thermocouple_start_c, acr_ohm
100%|██████████| 8/8 [00:00<00:00, 1468.66it/s]
2023-04-07 12:21:30,389 DEBUG [Loader:188] Dropping column cycle type from DataFrame
2023-04-07 12:21:30,391 DEBUG [Loader:188] Dropping column maccor_min_current_ma from DataFrame
2023-04-07 12:21:30,395 DEBUG [Loader:188] Dropping column maccor_min_voltage_mv from DataFrame
2023-04-07 12:21:30,396 DEBUG [Loader:188] Dropping column acr_ohm from DataFrame
2023-04-07 12:21:30,400 DEBUG [Loader:188] Dropping column dcir from DataFrame
2023-04-07 12:21:30,404 DEBUG [Loader:188] Dropping column maccor_charge_ther

8